In [ ]:
import ROOT
ROOT.RooMsgService.instance().setGlobalKillBelow(5)

In [ ]:
meas = ROOT.RooStats.HistFactory.Measurement( "meas", "meas" )
meas.SetPOI( "SignalStrength" )
meas.SetLumi( 1.0 )
meas.SetLumiRelErr( 0.02 )
meas.AddConstantParam( "Lumi" )

## Make some example data

expected and observed data, one bin, 10% more events observed than expected so SignalStrength should be 1.1

In [ ]:
data_hist = ROOT.TH1D("observed","observed",1,0,1)
for i in range(1100):
    data_hist.Fill(0.5)
signal_hist = ROOT.TH1D("above_expected","above_expected",1,0,1)
for i in range(100):
    signal_hist.Fill(0.5)
model_hist = ROOT.TH1D("expected","expected",1,0,1)
for i in range(1000):
    model_hist.Fill(0.5)

## Create a measurement  and fill it.

In [ ]:
chan = ROOT.RooStats.HistFactory.Channel( "Region1" )
chan.SetStatErrorConfig(0.05, "Poisson")
chan.SetData( data_hist )

In [ ]:
model = ROOT.RooStats.HistFactory.Sample( "model" )
model.SetNormalizeByTheory( False )
model.SetHisto( model_hist )

signal = ROOT.RooStats.HistFactory.Sample( "signal" )
signal.SetNormalizeByTheory( False )
signal.SetHisto( signal_hist )

And add our parameter of interest with a sensible bound.

In [ ]:
signal.AddNormFactor( "SignalStrength", 1, 0, 3)

and one nuisance parameter

In [ ]:
uncertainty_up   = 1000 * 1.1
uncertainty_down = 1000 * 0.9
signal.AddOverallSys( "signal_norm_uncertainty",  uncertainty_down*.1, uncertainty_up*.1 )
model.AddOverallSys( "background_norm_uncertainty",  uncertainty_down,uncertainty_up )

In [ ]:
sig_np_up = signal_hist.Clone()
sig_np_down = signal_hist.Clone()
bkg_np_up = model_hist.Clone()
bkg_np_down = model_hist.Clone()
for b in range(1,sig_np_up.GetNbinsX()+1):
    sig_np_up.SetBinContent(b, sig_np_up.GetBinContent(b) + sig_np_up.GetBinContent(b) * .1 * b)
    sig_np_down.SetBinContent(b, sig_np_down.GetBinContent(b) - sig_np_down.GetBinContent(b) * 0.1 * b)
    bkg_np_up.SetBinContent(b, bkg_np_up.GetBinContent(b) + bkg_np_up.GetBinContent(b) * .1 * b)
    bkg_np_down.SetBinContent(b, bkg_np_down.GetBinContent(b) - bkg_np_down.GetBinContent(b) * 0.1 * b)

In [ ]:
signal_shape = ROOT.RooStats.HistFactory.HistoSys("signal_shape")
signal_shape.SetHistoHigh( sig_np_up )
signal_shape.SetHistoLow( sig_np_down )
signal.AddHistoSys( signal_shape )

background_shape = ROOT.RooStats.HistFactory.HistoSys("background_shape")
background_shape.SetHistoHigh( bkg_np_up )
background_shape.SetHistoLow( bkg_np_down )
model.AddHistoSys( background_shape )

And add to measuremnet

In [ ]:
chan.AddSample( model )
chan.AddSample( signal )
meas.AddChannel( chan )

## Make workspace!

In [ ]:
hist2workspace = ROOT.RooStats.HistFactory.HistoToWorkspaceFactoryFast(meas)

In [ ]:
workspace = hist2workspace.MakeSingleChannelModel( meas, chan )

ok this was put into a function...

In [ ]:
from Builder import get_workspace

In [ ]:
workspace = get_workspace(nchannels = 1, events = 1000, nbins = 1)

ok that seemed to work

In [ ]:
workspace.SetName('BinnedWorkspace')
workspace.writeToFile("output/workspace{}channels{}events{}bins{}nps.root".format(1,1000,1,0))

In [ ]:
events = 1000
chans = 1
nps = 0
for bins in [1,10,20,30,40,50,60,70,80,90,100]:
    workspace = get_workspace(nchannels = chans, events = events, nbins = bins, nnps = nps)
    workspace.SetName('BinnedWorkspace')
    workspace.writeToFile("output/workspace{}channels{}events{}bins{}nps.root".format(chans, events, bins, nps))

In [ ]:
events = 1000
chans = 1
nps = 0
bins = 1
for events in [10,100,1000,10000,100000,1000000,10000000]:
    workspace = get_workspace(nchannels = chans, events = events, nbins = bins, nnps = nps)
    workspace.SetName('BinnedWorkspace')
    workspace.writeToFile("output/workspace{}channels{}events{}bins{}nps.root".format(chans, events, bins, nps))

In [ ]:
events = 1000
chans = 1
nps = 0
bins = 1
for nps in range(10):
    workspace = get_workspace(nchannels = chans, events = events, nbins = bins, nnps = nps)
    workspace.SetName('BinnedWorkspace')
    workspace.writeToFile("output/workspace{}channels{}events{}bins{}nps.root".format(chans, events, bins, nps))